# Project For Data Science Term 2

## Profit and Waste Analysis of Repeating Menu Systems in a Filipino Eatery

In [20]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pylatex


# data = pd.read_excel("filipino_eatery_month_dataset_adjusted_behavior_datasci.xlsx")
data=pd.read_excel("final_realism_upgraded_dataset.xlsx")
data

,Date,Day,Dish,Dish_Type,Cost_PHP,Price_PHP,Total_Serving,Sold,Unsold,Revenue,Profit
0,2025-01-06,Monday,Sinigang na Baboy,Soup,629,60,11,11,0,660,31
1,2025-01-06,Monday,Chicken Adobo,Chicken,645,60,15,12,3,720,75
2,2025-01-06,Monday,Pork Menudo,Pork,820,60,13,13,0,780,-40
3,2025-01-06,Monday,Sisig,Pork,810,70,14,14,0,980,170
4,2025-01-06,Monday,Ginataang Tulingan,Fish,370,60,12,10,2,600,230
...,...,...,...,...,...,...,...,...,...,...,...
170,2025-02-01,Saturday,Sisig,Pork,810,70,13,13,0,910,100
171,2025-02-01,Saturday,Pork Adobo,Pork,600,60,14,12,2,720,120
172,2025-02-01,Saturday,Ginataang Tulingan,Fish,370,60,15,13,2,780,410
173,2025-02-01,Saturday,Paksiw na Bangus,Fish,332,60,12,12,0,720,388


### Exploratory Data Analysis

In [21]:
data.info()

<class 'pandas.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   Date           175 non-null    str  
 1   Day            175 non-null    str  
 2   Dish           175 non-null    str  
 3   Dish_Type      175 non-null    str  
 4   Cost_PHP       175 non-null    int64
 5   Price_PHP      175 non-null    int64
 6   Total_Serving  175 non-null    int64
 7   Sold           175 non-null    int64
 8   Unsold         175 non-null    int64
 9   Revenue        175 non-null    int64
 10  Profit         175 non-null    int64
dtypes: int64(7), str(4)
memory usage: 21.1 KB


In [22]:
data.describe()

,Cost_PHP,Price_PHP,Total_Serving,Sold,Unsold,Revenue,Profit
count,175.000000,175.000000,175.000000,175.000000,175.000000,175.000000,175.000000
mean,554.828571,59.200000,13.251429,12.457143,0.794286,739.085714,184.257143
std,232.153819,11.164393,1.323825,1.342130,0.892479,171.682909,173.412793
min,229.000000,40.000000,10.000000,10.000000,0.000000,400.000000,-220.000000
25%,300.000000,50.000000,12.000000,11.500000,0.000000,600.000000,59.000000
50%,600.000000,60.000000,13.000000,12.000000,1.000000,720.000000,180.000000
75%,727.000000,60.000000,14.000000,14.000000,1.000000,840.000000,302.500000
max,943.000000,80.000000,15.000000,15.000000,3.000000,1200.000000,568.000000


In [23]:
data.dropna(inplace=True)

In [24]:
# aggregate by week
data['Date'] = pd.to_datetime(data['Date'])
data['Week'] = data['Date'].dt.isocalendar().week
data['Week'] = data['Week'] - data['Week'].min() + 1  # normalize to start at 1

weekly_summary = data.groupby('Week').agg(
    Cost_PHP=('Cost_PHP', 'sum'),
    Price_PHP=('Price_PHP', 'sum'),
    Total_Serving=('Total_Serving', 'sum'),
    Number_Sold=('Sold', 'sum'),
    Unsold_Quantity=('Unsold', 'sum'),
    Revenue=('Revenue', 'sum'),
    Profit=('Profit', 'sum'),
)

weekly_summary['Loss_PHP'] = (data['Price_PHP'] * data['Unsold']).groupby(data['Week']).sum()

weekly_summary

,Cost_PHP,Price_PHP,Total_Serving,Number_Sold,Unsold_Quantity,Revenue,Profit,Loss_PHP
Week,,,,,,,,
1,23604,2530,563,530,33,31240,7636,1880
2,22797,2510,557,533,24,31940,9143,1400
3,26631,2750,610,563,47,33740,7109,2680
4,24063,2570,589,554,35,32420,8357,2010


In [25]:
data.info()

<class 'pandas.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Date           175 non-null    datetime64[us]
 1   Day            175 non-null    str           
 2   Dish           175 non-null    str           
 3   Dish_Type      175 non-null    str           
 4   Cost_PHP       175 non-null    int64         
 5   Price_PHP      175 non-null    int64         
 6   Total_Serving  175 non-null    int64         
 7   Sold           175 non-null    int64         
 8   Unsold         175 non-null    int64         
 9   Revenue        175 non-null    int64         
 10  Profit         175 non-null    int64         
 11  Week           175 non-null    UInt32        
dtypes: UInt32(1), datetime64[us](1), int64(7), str(3)
memory usage: 20.3 KB


In [26]:
data.groupby('Dish').agg(
    Cost_PHP=('Cost_PHP', 'sum'),
    Revenue=('Revenue', 'sum'),
    Profit=('Profit', 'sum')
).reset_index()

,Dish,Cost_PHP,Revenue,Profit
0,Beef Steak,5906,7440,1534
1,Chicken Adobo,3225,3840,615
2,Chicken BBQ,5320,4850,-470
3,Chicken Curry,2100,2220,120
4,Chopsuey,751,1440,689
5,Dinakdakan,11316,12800,1484
6,Fried Salmon,3900,8150,4250
7,Fried Tilapia,3300,7860,4560
8,Ginataang Kalabasa,2005,4120,2115
9,Ginataang Tulingan,5180,10680,5500


In [27]:
data.groupby('Dish_Type').agg(
    Cost_PHP=('Cost_PHP', 'sum'),
    Revenue=('Revenue', 'sum'),
    Profit=('Profit', 'sum')
).reset_index()

,Dish_Type,Cost_PHP,Revenue,Profit
0,Beef,5906,7440,1534
1,Chicken,16461,16490,29
2,Fish,15700,34250,18550
3,Pork,38106,41980,3874
4,Soup,14857,17060,2203
5,Vegetable,6065,12120,6055


In [34]:
# Using pandas query method to filter fish dishes
fish_dishes = data.query("Dish == 'Dinakdakan'")
fish_dishes

,Date,Day,Dish,Dish_Type,Cost_PHP,Price_PHP,Total_Serving,Sold,Unsold,Revenue,Profit,Week
17,2025-01-08,Wednesday,Dinakdakan,Pork,943,80,14,13,1,1040,97,1
18,2025-01-08,Wednesday,Dinakdakan,Pork,943,80,14,13,1,1040,97,1
46,2025-01-13,Monday,Dinakdakan,Pork,943,80,12,12,0,960,17,2
52,2025-01-14,Tuesday,Dinakdakan,Pork,943,80,14,14,0,1120,177,2
60,2025-01-15,Wednesday,Dinakdakan,Pork,943,80,14,14,0,1120,177,2
73,2025-01-17,Friday,Dinakdakan,Pork,943,80,14,13,1,1040,97,2
74,2025-01-17,Friday,Dinakdakan,Pork,943,80,15,14,1,1120,177,2
103,2025-01-22,Wednesday,Dinakdakan,Pork,943,80,15,14,1,1120,177,3
127,2025-01-25,Saturday,Dinakdakan,Pork,943,80,13,13,0,1040,97,3
140,2025-01-28,Tuesday,Dinakdakan,Pork,943,80,14,13,1,1040,97,4


### latex

In [28]:
# from pylatex import Document, Section, Subsection, Command, NoEscape

# # -----------------------------
# # Create IEEE Document
# # -----------------------------
# doc = Document(documentclass="IEEEtran")

# # -----------------------------
# # Title Information
# # -----------------------------
# doc.preamble.append(Command('title',
#     'Profit and Waste Analysis of Repeating Menu Systems in a Filipino Eatery'))
# doc.preamble.append(Command('author', 'Author Name'))
# doc.append(NoEscape(r'\maketitle'))

# # -----------------------------
# # Abstract
# # -----------------------------
# with doc.create(Section("Abstract", numbering=False)):
#     doc.append(
#         "This study analyzes dish-level profitability and waste patterns "
#         "in a repeating menu system of a Filipino eatery. Using operational "
#         "batch data, revenue, profit, sell-through rates, and waste percentages "
#         "were computed to evaluate menu performance. Results highlight the "
#         "impact of menu composition and intra-menu competition on dish-level "
#         "outcomes. The findings demonstrate how descriptive data analytics can "
#         "support operational decision-making and improve production efficiency "
#         "in small-scale food enterprises."
#     )

# # =====================================================
# # CHAPTER 1 — INTRODUCTION
# # =====================================================
# with doc.create(Section("Introduction")):

#     with doc.create(Subsection("Background of the Study")):
#         doc.append(
#             "Small-scale food establishments often rely on experience-based "
#             "decision-making when selecting daily dish combinations and "
#             "production quantities. Repeating menu systems may produce "
#             "variability in profitability and unsold servings. This study "
#             "applies dish-level operational analytics to evaluate profit and "
#             "waste patterns in a structured menu cycle."
#         )

#     with doc.create(Subsection("Problem Statement")):
#         doc.append(
#             "Despite consistent menu rotation, variability in dish-level "
#             "profitability and unsold servings persists. The study investigates "
#             "how operational data can be used to evaluate and improve financial "
#             "performance while minimizing waste."
#         )

#     with doc.create(Subsection("Objectives of the Study")):
#         doc.append(
#             "1) Analyze dish-level profitability.\n"
#             "2) Measure waste using unsold quantity.\n"
#             "3) Compare performance across dish types.\n"
#             "4) Identify high and low performing dishes.\n"
#             "5) Provide data-driven menu recommendations."
#         )

#     with doc.create(Subsection("Scope and Limitations")):
#         doc.append(
#             "The study uses one month of operational batch data. "
#             "Customer-level transaction data and predictive modeling "
#             "are not included."
#         )

# # =====================================================
# # CHAPTER 2 — RELATED LITERATURE
# # =====================================================
# with doc.create(Section("Related Literature")):

#     with doc.create(Subsection("Menu Engineering")):
#         doc.append(
#             "Menu engineering evaluates dishes based on profitability "
#             "and sales performance to support pricing and production decisions."
#         )

#     with doc.create(Subsection("Food Waste in Food Service")):
#         doc.append(
#             "Food waste in small establishments is commonly caused by "
#             "demand uncertainty and overproduction. Sell-through rate is "
#             "a key operational metric."
#         )

#     with doc.create(Subsection("Data Analytics in Small Enterprises")):
#         doc.append(
#             "Descriptive and diagnostic analytics provide measurable insights "
#             "into operational efficiency and resource utilization."
#         )

# # =====================================================
# # CHAPTER 3 — METHODOLOGY
# # =====================================================
# with doc.create(Section("Methodology")):

#     with doc.create(Subsection("Research Design")):
#         doc.append(
#             "This study applies descriptive and diagnostic analytics to "
#             "evaluate dish-level performance in a repeating menu system."
#         )

#     with doc.create(Subsection("Dataset Description")):
#         doc.append(
#             "The dataset includes dish name, type, cost, price, total serving, "
#             "number sold, revenue, profit, and unsold quantity."
#         )

#     with doc.create(Subsection("Performance Metrics")):
#         doc.append(NoEscape(r"Revenue = Price $\times$ Number\_Sold \\"))
#         doc.append(NoEscape(r"Profit = Revenue - Cost \\"))
#         doc.append(NoEscape(r"SellThrough = Number\_Sold / Total\_Serving \\"))
#         doc.append(NoEscape(r"Waste\% = Unsold\_Quantity / Total\_Serving"))

#     with doc.create(Subsection("Analytical Procedure")):
#         doc.append(
#             "Profitability metrics were computed for each dish. "
#             "Dishes were ranked by profit and waste, aggregated by type, "
#             "and evaluated using a profit-waste performance matrix."
#         )

# # =====================================================
# # CHAPTER 4 — RESULTS AND DISCUSSION
# # =====================================================
# with doc.create(Section("Results and Discussion")):

#     with doc.create(Subsection("Profitability Analysis")):
#         doc.append(
#             "High-performing dishes contributed disproportionately "
#             "to total revenue, while certain dishes exhibited lower margins."
#         )

#     with doc.create(Subsection("Waste Analysis")):
#         doc.append(
#             "Waste levels varied across dish types, indicating production "
#             "imbalance and demand variability."
#         )

#     with doc.create(Subsection("Menu Composition Effects")):
#         doc.append(
#             "Chicken dishes exhibited increased unsold quantities when "
#             "served alongside multiple pork or beef dishes, indicating "
#             "intra-menu competition effects."
#         )

# # =====================================================
# # CHAPTER 5 — CONCLUSION
# # =====================================================
# with doc.create(Section("Conclusion")):
#     doc.append(
#         "Dish-level operational analytics provides measurable insights "
#         "into profitability and waste behavior. Menu composition influences "
#         "performance, and balancing dish types can improve efficiency. "
#         "Future work may incorporate predictive modeling and optimization."
#     )

# # -----------------------------
# # Generate .tex file
# # -----------------------------
# doc.generate_tex("profit_waste_ieee")
# print("LaTeX file generated: profit_waste_ieee.tex")

